In [1]:
import tensorflow_datasets as tfds
from tensorflow.keras.layers import *
from tensorflow.keras import Model
import tensorflow as tf

from sklearn.model_selection import train_test_split
import numpy as np

## Load data

In [2]:
dataset, info = tfds.load(
    "uc_merced",
    split=['train'],
    with_info=True,
    batch_size=-1

)

In [3]:
info

tfds.core.DatasetInfo(
    name='uc_merced',
    full_name='uc_merced/2.0.0',
    description="""
    UC Merced is a 21 class land use remote sensing image dataset, with 100 images
    per class. The images were manually extracted from large images from the USGS
    National Map Urban Area Imagery collection for various urban areas around the
    country. The pixel resolution of this public domain imagery is 0.3 m.
    
    While most images are 256x256 pixels, there are 44 images with different shape.
    """,
    homepage='http://weegee.vision.ucmerced.edu/datasets/landuse.html',
    data_path='dataset/uc_merced/2.0.0',
    file_format=tfrecord,
    download_size=317.07 MiB,
    dataset_size=238.63 MiB,
    features=FeaturesDict({
        'filename': Text(shape=(), dtype=string),
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=21),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
  

## Data processing

In [4]:
def preprocess(image):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (224, 224))
    return image

In [5]:
images = []
for i in tfds.as_numpy(dataset[0])[0]:
    images.append(preprocess(i))

images = np.array(images)
labels = tfds.as_numpy(dataset[0])[1].astype('uint32')

In [6]:
images.shape, labels.shape

((2100, 224, 224, 3), (2100,))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1890, 224, 224, 3), (210, 224, 224, 3), (1890,), (210,))

## Build model

In [8]:
def identity_block(x, filters):

    f1, f2, f3 = filters

    x1 = x

    x = Conv2D(filters=f1, kernel_size=(1, 1), strides=(1, 1))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=f2, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=f3, kernel_size=(1, 1), strides=(1, 1))(x)
    x = BatchNormalization()(x)

    x = Add()([x, x1])

    return x

In [9]:
def convolutional_block(x, filters, s=2):

    f1, f2, f3 = filters

    x1 = x

    x = Conv2D(filters=f1, kernel_size=(1, 1), strides=(s, s))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=f2, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=f3, kernel_size=(1, 1), strides=(1, 1))(x)
    x = BatchNormalization()(x)

    x1 = Conv2D(filters=f3, kernel_size=(1, 1), strides=(s, s))(x1)
    x1 = BatchNormalization()(x1)

    x = Add()([x, x1])
    X = Activation('relu')(x)

    return X

In [10]:
def ResNet50(input_shape=(224, 224, 3), num_classes=1000):

    x_input = Input(input_shape)

    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(x_input)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = convolutional_block(x, [64, 64, 256], s=1)
    x = identity_block(x, [64, 64, 256])
    x = identity_block(x, [64, 64, 256])

    x = convolutional_block(x, [128, 128, 512], s=2)
    x = identity_block(x, [128, 128, 512])
    x = identity_block(x, [128, 128, 512])
    x = identity_block(x, [128, 128, 512])

    x = convolutional_block(x, [256, 256, 1024], s=2)
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])

    x = convolutional_block(x, [512, 512, 2048], s=2)
    x = identity_block(x, [512, 512, 2048])
    x = identity_block(x, [512, 512, 2048])

    x = GlobalAveragePooling2D()(x)
    x_output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=x_input, outputs=x_output, name='ResNet50')

    return model

In [11]:
resnet_model = ResNet50(num_classes=21)
resnet_model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                          

In [12]:
resnet_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
resnet_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50
)

Epoch 1/50
60/60 [==============================] - 71s 386ms/step - loss: 3.6092 - accuracy: 0.2354 - val_loss: 34.7496 - val_accuracy: 0.0429
Epoch 2/50
60/60 [==============================] - 20s 326ms/step - loss: 1.9761 - accuracy: 0.4058 - val_loss: 6.7525 - val_accuracy: 0.0905
Epoch 3/50
60/60 [==============================] - 19s 324ms/step - loss: 2.0689 - accuracy: 0.3788 - val_loss: 37.0243 - val_accuracy: 0.1095
Epoch 4/50
60/60 [==============================] - 19s 324ms/step - loss: 1.8079 - accuracy: 0.4434 - val_loss: 5.6733 - val_accuracy: 0.1429
Epoch 5/50
60/60 [==============================] - 19s 325ms/step - loss: 1.6354 - accuracy: 0.5000 - val_loss: 3.9206 - val_accuracy: 0.2143
Epoch 6/50
60/60 [==============================] - 19s 319ms/step - loss: 1.5111 - accuracy: 0.5175 - val_loss: 3.9384 - val_accuracy: 0.3286
Epoch 7/50
60/60 [==============================] - 20s 325ms/step - loss: 1.4555 - accuracy: 0.5413 - val_loss: 8.1784 - val_accuracy: 0.28